CramerV is measure of association between two categorical variables
https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V
> This notebook is for benchmarking several different implementation
- current running on i5-8400 CPU, 


In [2]:
import numpy as np
import pandas as pd


generate data

In [3]:
# generate a random numpy matrix with two categorical variables

# Define the categories
animal = ['cat', 'dog', 'mouse']
color = ['red', 'blue', 'green']
size = 100

# Generate pd.Dataframe with two categorical variables
df = pd.DataFrame({
    'animal': np.random.choice(animal, size),
    'color': np.random.choice(color, size)
})
df.head(2)

,animal,color
0,dog,blue
1,mouse,red


In [4]:
df['animal'] = df['animal'].astype('category').cat.codes
df['color'] = df['color'].astype('category').cat.codes

method 1, consise method with scipy and pandas

In [5]:
from scipy.stats.contingency import association

/home/zihaog/miniconda3/envs/py310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
def cramer1(a, b):
    xtab = pd.crosstab(a, b)
    return association(xtab, method='cramer')
cramer1(df['animal'], df['color'])

0.16616579479866295

method 2, implemented like algorithm described in wiki

In [7]:
from scipy.stats import chi2_contingency

In [8]:
def cramer2(a, b ):
    xtab = pd.crosstab(a, b)
    chi2 = chi2_contingency(xtab)[0]
    return np.sqrt((chi2 / xtab.values.sum()) / min(xtab.shape[0] - 1, xtab.shape[1] - 1))
cramer2(df['animal'], df['color'])

0.16616579479866295

method 3, improve the xtab with numpy

In [9]:
# modifed based on 
# https://gist.github.com/alexland/d6d64d3f634895b9dc8e

def numpy_crosstab(a,b):
    uniq_vals_a, idx_a = np.unique(a, return_inverse=True)
    uniq_vals_b, idx_b = np.unique(b, return_inverse=True)
    shape_xt = (uniq_vals_a.size, uniq_vals_b.size)
    xt = np.zeros(shape_xt, dtype='uint')
    np.add.at(xt, (idx_a, idx_b), 1)
    return xt
    
def cramer3(a, b ):
    xtab = numpy_crosstab(a, b)
    chi2 = chi2_contingency(xtab)[0]
    return np.sqrt((chi2 / xtab.sum()) / min(xtab.shape[0] - 1, xtab.shape[1] - 1))
cramer3(df['animal'], df['color'])

0.16616579479866295

method 4, futher improve xtab with numba JIT

In [10]:
import numba

In [11]:
@numba.jit(nopython=True)
def custom_np_unqiue_with_inverse(ar):
    '''
    the simplifed version of np.unique with return_inverse 
    # https://github.com/numpy/numpy/blob/a60de40f14580078dcfd9d0faf33ba3ec768fc8a/numpy/lib/_arraysetops_impl.py#L336
    becuase we only need the return_inverse
    And we know we will only have 1-d array without any nan and it will be integer.
    '''
    ar = np.ascontiguousarray(ar)
    perm = ar.argsort(kind = 'quicksort')
    aux = ar[perm]
    # get unique
    mask = np.empty(aux.shape, dtype=np.bool_)
    mask[:1] = True
    mask[1:] = aux[1:] != aux[:-1]
    ret = aux[mask]
    # get the indices of the unique array
    imask = np.cumsum(mask) - 1
    inv_idx = np.empty(mask.shape, dtype=np.intp)
    inv_idx[perm] = imask
    return ret, inv_idx
@numba.jit(nopython=True)
def numba_crosstab(a,b):
    uniq_vals_a, idx_a = custom_np_unqiue_with_inverse(a)
    uniq_vals_b, idx_b = custom_np_unqiue_with_inverse(b)
    shape_xt = (uniq_vals_a.size, uniq_vals_b.size)
    xt = np.zeros(shape_xt, dtype='uint')
    # equivalent to np.add.at(xt, (idx_a, idx_b), 1)
    for i in range(len(idx_a)):
        xt[idx_a[i], idx_b[i]] += 1
    return xt

In [12]:
def cramer4(a, b ):
    xtab = numba_crosstab(a, b)
    chi2 = chi2_contingency(xtab)[0]
    return np.sqrt((chi2 / xtab.sum()) / min(xtab.shape[0] - 1, xtab.shape[1] - 1))
cramer4(df['animal'].values, df['color'].values)

0.16616579479866295

method 5, chi2_contingency has lots of features that we do no need, for example, no need to calculate p value for cramerv

In [ ]:
def cramer5(a, b ):
    xtab = numba_crosstab(a, b)
    chi2 = chi2_contingency(xtab)[0]
    return np.sqrt((chi2 / xtab.sum()) / min(xtab.shape[0] - 1, xtab.shape[1] - 1))
cramer4(df['animal'].values, df['color'].values)

In [50]:
# modifed based on https://github.com/scipy/scipy/blob/main/scipy/stats/contingency.py
@numba.jit(nopython=True)
def margins_2d(a):
    margsum_row = x.sum(axis = 1)
    margsum_col = x.sum(axis = 0)
    return margsum_row, margsum_col

from functools import reduce
def expected_freq(observed):
    # Typically `observed` is an integer array. If `observed` has a large
    # number of dimensions or holds large values, some of the following
    # computations may overflow, so we first switch to floating point.
    observed = np.asarray(observed, dtype=np.float64)

    # Create a list of the marginal sums.
    margsums = margins_2d(observed)
    # Create the array of expected frequencies.  The shapes of the
    # marginal sums returned by apply_over_axes() are just what we
    # need for broadcasting in the following product.
    d = observed.ndim
    expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)
    return expected    

In [51]:
expected_freq(numba_crosstab(df['animal'].values, df['color'].values))

array([12.95, 11.84,  8.58])

In [24]:
@numba.jit(nopython=True)
def numba_expected_freq(observed):
    # Typically `observed` is an integer array. If `observed` has a large
    # number of dimensions or holds large values, some of the following
    # computations may overflow, so we first switch to floating point.
    observed = np.asarray(observed, dtype=np.float64)

    # Create a list of the marginal sums.
    margsums = margins(observed)

In [25]:
numba_expected_freq(numba_crosstab(df['animal'].values, df['color'].values))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mUse of unsupported NumPy function 'numpy.apply_over_axes' or unsupported use of the function.
[1m
File "../../../../../tmp/ipykernel_25241/7101530.py", line 6:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m
[0m[1mDuring: typing of get attribute at /tmp/ipykernel_25241/7101530.py (6)[0m
[1m
File "../../../../../tmp/ipykernel_25241/7101530.py", line 6:[0m
[1m<source missing, REPL/exec in use?>[0m

[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function margins at 0x7f02b32b0550>))[0m
[0m[1mDuring: typing of call at /tmp/ipykernel_25241/2431546293.py (9)
[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function margins at 0x7f02b32b0550>))[0m
[0m[1mDuring: typing of call at /tmp/ipykernel_25241/2431546293.py (9)
[0m
[1m
File "../../../../../tmp/ipykernel_25241/2431546293.py", line 9:[0m


benchmarking

In [30]:
%%timeit
cramer1(df['animal'], df['color'])

5.88 ms ± 390 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%%timeit
cramer2(df['animal'], df['color'])

5.37 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%%timeit
cramer3(df['animal'], df['color'])

323 µs ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
%%timeit
cramer4(df['animal'].values, df['color'].values)

243 µs ± 8.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%%timeit
numba_crosstab(df['animal'].values, df['color'].values)

10.3 µs ± 99.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
%%timeit
a = df['color'].values

2.42 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
